## Imports and Setup
Updated with NOISE_STD, VGG_WEIGHT, and library imports. Checks for GPU availability

In [ ]:
import os
import json
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, random_split
from torch.amp import autocast, GradScaler 
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Set seeds
torch.manual_seed(42)
np.random.seed(42)

# --- Configuration ---
BATCH_SIZE = 4
LEARNING_RATE = 1e-4
EPOCHS = 100
OUTPUT_DIR = "/kaggle/working"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")

# Dataset Discovery
search_path = "/kaggle/input/**/*.pt"
pt_files = glob.glob(search_path, recursive=True)
if not pt_files:
    raise FileNotFoundError("❌ Dataset not found! Make sure to upload the processed .pt file.")
DATA_FILE = pt_files[0]
DATA_DIR = os.path.dirname(DATA_FILE)
print(f"📂 Dataset found at: {DATA_FILE}")

## Dataset Class 
Added ```noise_std``` parameter for Sim-to-Real gap augmentation.

In [ ]:
class FluidLoader(Dataset):
    def __init__(self, pt_file):
        print(f"⏳ Loading data map...")
        try:
            # mmap=True is crucial for Kaggle RAM limits
            data = torch.load(pt_file, map_location='cpu', mmap=True)
        except:
            print("⚠️ mmap failed. Loading to RAM.")
            data = torch.load(pt_file, map_location='cpu')
            
        self.inputs = data['inputs']
        self.targets = data['targets']
        
        # Load all normalization constants
        # Fallback to 1.0 if not found (legacy support)
        self.K_vel = float(data.get('K_vel', 1.0))
        self.K_pres = float(data.get('K_pres', 1.0))
        self.K_smoke = float(data.get('K_smoke', 1.0))
            
        print(f"✅ Loaded {len(self.inputs)} samples.")
        print(f"   K_vel: {self.K_vel} | K_pres: {self.K_pres} | K_smoke: {self.K_smoke}")

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        # Data is already processed (Normalized & Upscaled) in the .pt file
        # We just convert to float32 for training
        lr = self.inputs[idx].float()
        hr = self.targets[idx].float()
        return lr, hr

## Model Architecture (ResUNet)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(residual)
        out = self.relu(out)
        return out

class ResUNet(nn.Module):
    def __init__(self, in_channels=4, out_channels=4, features=[64, 128, 256, 512]):
        super(ResUNet, self).__init__()
        self.encoder = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Encoder
        self.input_conv = nn.Sequential(
            nn.Conv2d(in_channels, features[0], kernel_size=3, padding=1),
            nn.BatchNorm2d(features[0]),
            nn.ReLU(inplace=True)
        )
        input_feat = features[0]
        for feature in features:
            self.encoder.append(ResidualBlock(input_feat, feature))
            input_feat = feature

        # Bottleneck
        self.bottleneck = ResidualBlock(features[-1], features[-1] * 2)

        # Decoder
        self.upconvs = nn.ModuleList()
        self.decoder = nn.ModuleList()
        features = features[::-1]
        for feature in features:
            self.upconvs.append(nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2))
            self.decoder.append(ResidualBlock(feature * 2, feature))

        # Output
        self.final_conv = nn.Conv2d(features[-1], out_channels, kernel_size=1)
        
    def forward(self, x):
        skip_connections = []
        out = self.input_conv(x)
        
        for layer in self.encoder:
            out = layer(out)
            skip_connections.append(out)
            out = self.pool(out)
            
        out = self.bottleneck(out)
        skip_connections = skip_connections[::-1] 
        
        for idx in range(len(self.decoder)):
            out = self.upconvs[idx](out)
            skip = skip_connections[idx]
            if out.shape != skip.shape:
                out = F.interpolate(out, size=skip.shape[2:], mode='bilinear', align_corners=False)
            concat_skip = torch.cat((skip, out), dim=1)
            out = self.decoder[idx](concat_skip)
            
        out = self.final_conv(out)
        return out

## Loss Functions ( VGG, Physics Informed)

In [ ]:
class DivergenceLoss(nn.Module):
    def __init__(self):
        super(DivergenceLoss, self).__init__()

    def forward(self, output, K_vel):
        # Extract Velocity Channels (0, 1)
        u = output[:, 0, :, :] * K_vel
        v = output[:, 1, :, :] * K_vel
        
        # Calculate Divergence: du/dx + dv/dy
        du_dx = u[:, :, 1:] - u[:, :, :-1]
        dv_dy = v[:, 1:, :] - v[:, :-1, :]
        
        # Crop to align dimensions
        return torch.mean((du_dx[:, :-1, :] + dv_dy[:, :, :-1])**2)

class PressurePoissonLoss(nn.Module):
    def __init__(self):
        super(PressurePoissonLoss, self).__init__()
        # Laplacian kernel
        self.register_buffer('laplacian', torch.tensor([[[[0, 1, 0], [1, -4, 1], [0, 1, 0]]]], dtype=torch.float32))
        # Sobel kernels for gradients
        self.register_buffer('k_x', torch.tensor([[[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]]], dtype=torch.float32) / 8.0)
        self.register_buffer('k_y', torch.tensor([[[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]]], dtype=torch.float32) / 8.0)

    def get_grads(self, f):
        return F.conv2d(f, self.k_x, padding=1), F.conv2d(f, self.k_y, padding=1)

    def get_laplacian(self, f):
        return F.conv2d(f, self.laplacian, padding=1)

    def get_divergence(self, fx, fy):
        dfx_dx, _ = self.get_grads(fx)
        _, dfy_dy = self.get_grads(fy)
        return dfx_dx + dfy_dy

    def forward(self, pred, K_vel, K_pres, K_smoke):
        # Unpack and Un-normalize
        u = pred[:, 0:1, :, :] * K_vel
        v = pred[:, 1:2, :, :] * K_vel
        p = pred[:, 2:3, :, :] * K_pres
        s = pred[:, 3:4, :, :] * K_smoke

        # 1. LHS: Laplacian of Pressure
        lhs = self.get_laplacian(p)

        # 2. RHS Term A: Divergence of Convection -(u.grad)u
        du_dx, du_dy = self.get_grads(u)
        dv_dx, dv_dy = self.get_grads(v)
        
        # Convection vector components
        conv_u = u * du_dx + v * du_dy 
        conv_v = u * dv_dx + v * dv_dy 
        div_convection = self.get_divergence(conv_u, conv_v)

        # 3. RHS Term B: Divergence of Buoyancy Force
        # Force ~ (0, s * 0.3)
        force_y = s * 0.3 
        div_force = self.get_divergence(torch.zeros_like(s), force_y)

        # Poisson Equation: Del^2 P = -Div(Conv) + Div(Force)
        rhs = -div_convection + div_force

        return F.mse_loss(lhs, rhs)

## Training Loop

In [ ]:
def train(use_physics=False, mse_weight=1.0, divergence_weight=0.1, ppe_weight=0.1):
    """
    Args:
        use_physics (bool): Whether to enable Divergence and PPE losses.
        mse_weight (float): Weight for Pixel Loss (MSE).
        divergence_weight (float): Weight for Divergence Loss (Mass Conservation).
        ppe_weight (float): Weight for Pressure Poisson Loss (Navier-Stokes).
    """
    # 1. Data
    dataset = FluidLoader(DATA_FILE)
    train_sz = int(0.8 * len(dataset))
    val_sz = len(dataset) - train_sz
    train_ds, val_ds = random_split(dataset, [train_sz, val_sz], generator=torch.Generator().manual_seed(42))
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    # 2. Setup (4 Inputs -> 4 Outputs)
    model = ResUNet(in_channels=4, out_channels=4).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    scaler = GradScaler('cuda') 
    
    # Loss Functions
    mse_fn = nn.MSELoss()
    div_fn = DivergenceLoss().to(device)
    ppe_fn = PressurePoissonLoss().to(device)
    
    # Normalization Constants
    K_vel = dataset.K_vel
    K_pres = dataset.K_pres
    K_smoke = dataset.K_smoke
    
    mode_name = "PINN_NS" if use_physics else "Baseline"
    print(f"\n🚀 Starting {mode_name} Training...")
    print(f"   > Weights: MSE={mse_weight} | Div={divergence_weight} | PPE={ppe_weight}")
    
    hist = {'train': [], 'val': [], 'lr': []}
    best_val_loss = float('inf')
    patience = 7
    counter = 0

    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0.0
        
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False)
        
        for x, y in loop:
            x, y = x.to(device), y.to(device)
            
            optimizer.zero_grad(set_to_none=True) 
            
            with autocast('cuda'): 
                pred = model(x)
                
                # --- Weighted Loss Calculation ---
                loss = 0.0
                
                # 1. MSE Loss (Pixel Accuracy)
                if mse_weight > 0:
                    loss += mse_weight * mse_fn(pred, y)
                
                # 2. Physics Losses
                if use_physics:
                    # Divergence (Mass)
                    if divergence_weight > 0:
                        loss += divergence_weight * div_fn(pred, K_vel)
                    
                    # Navier-Stokes (Momentum via Pressure)
                    if ppe_weight > 0:
                        loss += ppe_weight * ppe_fn(pred, K_vel, K_pres, K_smoke)
            
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            loop.set_postfix(loss=loss.item())
            
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                with autocast('cuda'):
                    pred = model(x)
                    # Track MSE for early stopping
                    val_loss += mse_fn(pred, y).item()
        
        avg_train = train_loss / len(train_loader)
        avg_val = val_loss / len(val_loader)
        
        scheduler.step(avg_val)
        current_lr = optimizer.param_groups[0]['lr']
        
        hist['train'].append(avg_train)
        hist['val'].append(avg_val)
        hist['lr'].append(current_lr)
        
        print(f"Epoch {epoch+1} | Train: {avg_train:.6f} | Val: {avg_val:.6f} | LR: {current_lr:.2e}")
        
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_val,
        }
        
        if avg_val < best_val_loss:
            best_val_loss = avg_val
            counter = 0
            torch.save(checkpoint, f"{OUTPUT_DIR}/{mode_name}_best.pth")
            print("  --> New Best Model Saved!")
        else:
            counter += 1
            if counter >= patience:
                print("🛑 Early Stopping triggered.")
                break
                
    # Plotting
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(hist['train'], label='Train')
    plt.plot(hist['val'], label='Val')
    plt.title(f'{mode_name} Loss')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(hist['lr'], color='orange')
    plt.title('Learning Rate')
    plt.grid(True, alpha=0.3)
    
    plt.savefig(f"{OUTPUT_DIR}/{mode_name}_history.png")
    plt.show()
    
    best_checkpoint = torch.load(f"{OUTPUT_DIR}/{mode_name}_best.pth")
    model.load_state_dict(best_checkpoint['model_state_dict'])
    
    return model, dataset

## Execute
Run this cell to start.

In [ ]:
# Train with Navier-Stokes physics
model, ds = train(
    use_physics=True, 
    mse_weight=1.0, 
    divergence_weight=0.1, 
    ppe_weight=0.1
)